In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import Client, SSHCluster
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

# Macro-Pipeline AHN4 Workflow - Feature Extraction (Non-Ground Points)

## Set Run-Specific Input

Choose whether you want to run all input files or run the only input files listed in `filename`.

In [2]:
path_root = pathlib.Path('/project/lidarac/Software/Yifang/JupyterDaskOnSLURM/AHN4_test/')

# path to normalized files 
path_input = path_root / 'Normalized'

# path to targets
path_output = path_input.parent / 'Targets_veg'

run = 'all'  # 'all', 'from_file'
#filename = 'feature_extraction_non-ground_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'from_file']

In [3]:
tiles = [el for el in path_input.iterdir() if el.match('tile_*_*.laz')]
print('Found: {} tiles'.format(len(tiles)))
if run == 'from_file':
    with open(filename, 'r') as f:
        tiles_read = json.load(f)
    tiles_read = [path_input/f for f in tiles_read]
    # check whether all files are available on dCache
    assert all([f in tiles for f in tiles_read]), f'Some of the tiles in {filename} are not in input dir'
    tiles = tiles_read
print('Retrieve and extract features for: {} tiles'.format(len(tiles)))

Found: 4 tiles
Retrieve and extract features for: 4 tiles


## Setup Cluster

Setup Dask cluster used for all the macro-pipeline calculations.

In [ ]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 2  

# start the cluster
scheduler_node = 'node1'
hosts = [f'node{i}' for i in range(1, 11)]
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 0,
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

In [ ]:
client = Client("tcp://10.0.2.186:41037")
client

## Feature Extraction

We extract features for the only non-ground points.

In [7]:
# details of the retiling schema
grid = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

# target mesh size
tile_mesh_size = 10.

# list of features
z = "normalized_height" # most of the features are based on the normalized height
features = ["point_density", "sigma_z"] 
features += [f"perc_{percentile}_{z}" for percentile in (25, 50, 75, 95)]
features += [f"{feature}_{z}" for feature in ("max",
                                              "median",
                                              "mean", 
                                              "var", 
                                              "std", 
                                              "skew", 
                                              "kurto", 
                                              "coeff_var", 
                                              "entropy", 
                                              "density_absolute_mean")]
#features += [f"band_ratio_{z}<5", f"band_ratio_5<{z}<20", f"band_ratio_20<{z}"]
features += [f"eigenv_{i}" for i in range(1, 4)]
features += [f"normal_vector_{i}" for i in range(1, 4)]

# define custom feature extractors
custom_feature_extractors = [{'extractor_name': 'BandRatioFeatureExtractor', 
                              'data_key': z, 
                              'lower_limit': low_lim,        
                              'upper_limit': up_lim} for low_lim, up_lim in ((None, 5), (5, 20), (20, None))]

# setup input dictionary to configure the feature extraction pipeline
feature_extraction_input_non_ground = {
    'setup_local_fs': {'input_folder': path_input.as_posix(),
                       'output_folder': path_output.as_posix()},
    'load': {'attributes': ['raw_classification', 'normalized_height']}, 
    'add_custom_features': {
        'custom_feature_list': custom_feature_extractors
    },
    'apply_filter': {
        'filter_type': 'select_equal',
        'attribute': 'raw_classification',
        #unclassified (1), ground (2), buildings (6), water (9), wire conductor (14), artificial objects (26), never classified (0)
        'value': 1
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        'validate_precision': 1.e-3,  # solves numerical issues for 6 tiles which have points on the edge
        **grid
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False,
        'overwrite': True
    },
    'clear_cache' : {},
}

# write input dictionary to JSON file
with open('feature_extraction_non-ground.json', 'w') as f:
    json.dump(feature_extraction_input_non_ground, f)

In [8]:
macro = MacroPipeline()

# extract the tile indices from the tile names
tile_indices = [[int(el) for el in tile.name.split('.')[0].split('_')[1:]] for tile in tiles]

# add pipeline list to macro-pipeline object and set the corresponding labels
macro.tasks = [DataProcessing(t.name, tile_index=idx).config(feature_extraction_input_non_ground) 
               for t, idx in zip(tiles, tile_indices)]
macro.set_labels([os.path.splitext(tile.name)[0] for tile in tiles])

macro.setup_cluster(cluster="tcp://10.0.2.186:41037")

# run!
macro.run()

# save outcome results and write name of failed pipelines to file
macro.print_outcome(to_file='feature_extraction_non-ground.out')
failed = macro.get_failed_pipelines()
if failed:
    with open('feature_extraction_non-ground_failed.json', 'w') as f:
        json.dump(['.'.join([pip.label, 'laz']) for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

## Terminate cluster

In [7]:
macro.shutdown()

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://145.100.59.123:8786' processes=20 threads=20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1157, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/usr/local/lib/python3.7/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://145.100.59.123:8786' processes=20 threads=20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1157, in _heartbeat
   

## Troubleshooting 

### Cancel all jobs and restart the notebook

Copy and paste these lines in a separate Python shell. If the Dask dashboard shows that some tasks are still queued to be processed, run the lines again - this should clear the scheduler up and give back control to the current notebook. Normally proceed to terminate the cluster and restart the notebook.

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)